In [1]:
#### import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import Sequence
import pandas as pd
import os 
tf.compat.v1.enable_v2_behavior
# Import tensornetwork
import tensornetwork as tn
# Set the backend to tesorflow
# (default is numpy)
tn.set_default_backend("tensorflow")
batch_size = 64
class ClutteredMNISTDataset(Sequence):
    reg_dataset_size = 11276

    def __init__(self, base_path, csv_path, data_scaling=1., num_examples=None, balance=None, num_classes=2):
        self.base_path = base_path
        self.csv_path = csv_path
        self.csv = pd.read_csv(csv_path)
        self.data_scaling = data_scaling
        self.num_examples = num_examples
        self.balance = balance
        self.num_classes = num_classes
        self.img_paths = self.csv['img_path'].values
        self.lbls = self.csv['label'].values.astype(np.int32)
        self.weights = np.ones([len(self.img_paths), ])

        if self.num_examples is not None and self.balance is not None:
            # only rebalance if examples and balance is given
            assert self.num_examples <= len(self.img_paths), \
                'not enough examples in dataset {} - {}'.format(self.num_examples, len(self.img_paths))

            pos_num = int(self.balance * self.num_examples)
            neg_num = self.num_examples - pos_num

            pos_mask = (self.lbls == 1)
            pos_paths = self.img_paths[pos_mask][:pos_num]

            neg_mask = (self.lbls == 0)
            neg_paths = self.img_paths[neg_mask][:neg_num]

            self.img_paths = np.concatenate([pos_paths, neg_paths], 0)
            self.lbls = np.concatenate([np.ones([pos_num, ]), np.zeros([neg_num, ])], 0)
            self.weights = np.ones([self.num_examples, ])
            self.weights[:pos_num] /= pos_num
            self.weights[pos_num:] /= neg_num

        self.shrinkage = self.reg_dataset_size // len(self.img_paths)

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        path = os.path.join(self.base_path, self.img_paths[index])
        lbl = self.lbls[index].astype(np.int64)
        lbl = np.eye(self.num_classes)[lbl].astype(np.int64)
        img = np.load(path)
        if isinstance(img, np.lib.npyio.NpzFile):
            img = img['arr_0']

        if self.data_scaling != 1.:
            img = zoom(img, self.data_scaling)
            img = img.clip(0., 1.)

        img = (img[:,:,np.newaxis].astype(np.float32).astype(np.float32) -0.5)/1

        return img, lbl
    

class Batcher(Sequence):
    """Assemble a sequence of things into a sequence of batches."""
    def __init__(self, sequence, batch_size=16):
        self._batch_size = batch_size
        self._sequence = sequence
        self._idxs = np.arange(len(self._sequence))

    def __len__(self):
        return int(np.ceil(len(self._sequence) / self._batch_size))

    def __getitem__(self, i):
        if i >= len(self):
            raise IndexError("Index out of bounds")

        start = i*self._batch_size
        end = min(len(self._sequence), start+self._batch_size)
        data = [self._sequence[j] for j in self._idxs[start:end]]
        inputs = [d[0] for d in data]
        outputs = [d[1] for d in data]

        return self._stack(inputs), self._stack(outputs)

    def _stack(self, data):
        if data is None:
            return None

        if not isinstance(data[0], (list, tuple)):
            return np.stack(data)

        seq = type(data[0])
        K = len(data[0])
        data = seq(
            np.stack([d[k] for d in data])
            for k in range(K)
        )

        return data

    def on_epoch_end(self):
        np.random.shuffle(self._idxs)
        self._sequence.on_epoch_end()
        
        
        
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.enable_v2_behavior
# Import tensornetwork
import tensornetwork as tn
from tensorflow.keras.utils import to_categorical
tn.set_default_backend("tensorflow")




class GridMERAin16(tf.keras.layers.Layer):
    
    def __init__(self, kernel_dims, bond_dims, output_dims):
        super(GridMERAin16, self).__init__()
        # Create the variables for the layer.
        # In this case, the input tensor is (, 1936), we factorize it into a tensor (, 11, 11, 16)
        # first_dim: output shape?
        # second_dim: connect with data tensor
        # third_dim: inter-connect
        in_dims = int((kernel_dims//4)**2)
        self.entanglers = []
        self.isometries= []
        self.kernel_dims = kernel_dims
        self.output_dims = output_dims
        #entanglers
        self.entanglers1 = tf.Variable(tf.random.normal
                                             (shape=(in_dims, in_dims, 
                                                     in_dims, in_dims, bond_dims, bond_dims, bond_dims, bond_dims),
                                              stddev=1.0/10000), 
                                              trainable=True)
        self.entanglers2 = tf.Variable(tf.random.normal
                                             (shape=(bond_dims, bond_dims, 
                                                     bond_dims, bond_dims, bond_dims, bond_dims, bond_dims, bond_dims),
                                              stddev=1.0/10000), 
                                              trainable=True)
        # isometries
        self.isometries1 = [tf.Variable(tf.random.normal(shape=(in_dims, in_dims, in_dims, 
                                                                            bond_dims, bond_dims)
                                                                     , stddev=1.0/100000),
                                            trainable=True), 
                           tf.Variable(tf.random.normal(shape=(in_dims, in_dims, bond_dims, 
                                                                            in_dims, bond_dims)
                                                                     , stddev=1.0/100000),
                                            trainable=True),
                           tf.Variable(tf.random.normal(shape=(in_dims, bond_dims, in_dims, 
                                                                            in_dims, bond_dims)
                                                                     , stddev=1.0/100000),
                                            trainable=True),
                           tf.Variable(tf.random.normal(shape=(bond_dims, in_dims, in_dims, 
                                                                            in_dims, bond_dims)
                                                                     , stddev=1.0/100000),
                                            trainable=True)]
        
        self.isometries2 = tf.Variable(tf.random.normal(shape=(bond_dims, bond_dims, bond_dims, 
                                                                            bond_dims, output_dims)
                                                                     , stddev=1.0/100000),
                                            trainable=True)

        #print(self.final_mps.shape)
        self.bias = tf.Variable(tf.zeros(shape=(output_dims,)), name="bias", trainable=True)


    def call(self, inputs):
        # Define the contraction.
        # We break it out so we can parallelize a batch using tf.vectorized_map.
        def f(input_vec, entanglers1, entanglers2, isometries1, isometries2, bias_var, kernel_dims):
            input_vv = []
            step = int(kernel_dims//4)
            for i in range(4):
                for ii in range(4):
                    input_vv.append(tf.reshape(input_vec[i*step:i*step+step, ii*step:ii*step+step, 0], (1, step**2)))
            input_vec = tf.concat(input_vv, axis=0)
            input_vec = tf.reshape(input_vec, (16, step**2))
            input_vec = tf.unstack(input_vec)
            input_nodes = []
            for e_iv in input_vec:
                input_nodes.append(tn.Node(e_iv))
            
            e_nodes1 = tn.Node(entanglers1)
            e_nodes2 = tn.Node(entanglers2)
                
                                     
            isometries_nodes1 = []
            for eiso in isometries1:
                isometries_nodes1.append(tn.Node(eiso))
            isometries_nodes2 = tn.Node(isometries2)
            
            
            e_nodes1[0] ^ input_nodes[5][0]
            e_nodes1[1] ^ input_nodes[6][0]
            e_nodes1[2] ^ input_nodes[9][0]
            e_nodes1[3] ^ input_nodes[10][0]

            e_nodes1[4] ^ isometries_nodes1[0][3]
            e_nodes1[5] ^ isometries_nodes1[1][2]
            e_nodes1[6] ^ isometries_nodes1[2][1]
            e_nodes1[7] ^ isometries_nodes1[3][0]     
            
            input_nodes[0][0] ^ isometries_nodes1[0][0]
            input_nodes[1][0] ^ isometries_nodes1[0][1]
            input_nodes[4][0] ^ isometries_nodes1[0][2]
            
            input_nodes[2][0] ^ isometries_nodes1[1][0]
            input_nodes[3][0] ^ isometries_nodes1[1][1]
            input_nodes[7][0] ^ isometries_nodes1[1][3]
            
            input_nodes[8][0] ^ isometries_nodes1[2][0]
            input_nodes[12][0] ^ isometries_nodes1[2][2]
            input_nodes[13][0] ^ isometries_nodes1[2][3]
            
            input_nodes[11][0] ^ isometries_nodes1[3][1]
            input_nodes[14][0] ^ isometries_nodes1[3][2]
            input_nodes[15][0] ^ isometries_nodes1[3][3]
            
            
            isometries_nodes1[0][4] ^ e_nodes2[0]
            isometries_nodes1[1][4] ^ e_nodes2[1]
            isometries_nodes1[2][4] ^ e_nodes2[2]
            isometries_nodes1[3][4] ^ e_nodes2[3]

            e_nodes2[4] ^ isometries_nodes2[0]
            e_nodes2[5] ^ isometries_nodes2[1]
            e_nodes2[6] ^ isometries_nodes2[2]
            e_nodes2[7] ^ isometries_nodes2[3]

                            
            nodes = tn.reachable(isometries_nodes2)
            result = tn.contractors.greedy(nodes)
            result = result.tensor
            #print(result)
            #result = (c @ b).tensor
            # Finally, add bias.
            return result + bias_var

        # To deal with a batch of items, we can use the tf.vectorized_map function.
        # https://www.tensorflow.org/api_docs/python/tf/vectorized_map
        output = tf.vectorized_map(lambda vec: f(vec, self.entanglers1, self.entanglers2,
                                                 self.isometries1,  self.isometries2, self.bias, self.kernel_dims), inputs)
        return tf.reshape(output, (-1, self.output_dims))
    
from tensorflow.keras.layers import Lambda, Input, Concatenate, Reshape, Softmax, Dense, Flatten
from tensorflow.keras.models import Model, Sequential

def get_model(input_shape=(64, 64, 1)):
    x_in = Input(shape=input_shape)
    x_out_list = []
    for i in range(8):
        for j in range(8):
            subx = Lambda(lambda x:x[:, 8*i:8*(i+1),8*j:8*(j+1),:] )(x_in)
            x_out_list.append(GridMERAin16(kernel_dims=8, bond_dims=2, output_dims=1)(subx))
    x_out = Concatenate(axis=1)(x_out_list)
    x_out = Reshape(target_shape=(8, 8, 1))(x_out)
    y = GridMERAin16(kernel_dims=8, bond_dims=2, output_dims=2)(x_out)
    y = Softmax()(y)
    return Model(inputs=x_in, outputs=y)

def get_dense_model(input_shape=(64, 64, 1)):
    x_in = Input(shape=input_shape)
    x_out_list = []
    for i in range(8):
        for j in range(8):
            subx = Lambda(lambda x:x[:, 8*i:8*(i+1),8*j:8*(j+1),:] )(x_in)
            x_out_list.append(GridMERAin16(kernel_dims=8, bond_dims=2, output_dims=2)(subx))
    x_out = Concatenate(axis=1)(x_out_list)
    x_out = Flatten()(x_out)
    y = Dense(2, activation='softmax')(x_out)
    #x_out = Reshape(target_shape=(8, 8, 1))(x_out)
    #y = GridMERAin16(kernel_dims=8, bond_dims=2, output_dims=2)(x_out)
    #y = Softmax()(y)
    return Model(inputs=x_in, outputs=y)

def get_loop_model(input_shape=(64, 64, 1)):
    x_in = Input(shape=input_shape)
    x_out_list = []
    MERA_layer = GridMERAin16(kernel_dims=16, bond_dims=2, output_dims=2)
    for i in range(4):
        for j in range(4):
            subx = Lambda(lambda x:x[:, 16*i:16*(i+1),16*j:16*(j+1),:] )(x_in)
            x_out_list.append(MERA_layer(subx))
    x_out = Concatenate(axis=1)(x_out_list)
    x_out = Flatten()(x_out)
    y = Dense(2, activation='softmax')(x_out)
    #x_out = Reshape(target_shape=(8, 8, 1))(x_out)
    #y = GridMERAin16(kernel_dims=8, bond_dims=2, output_dims=2)(x_out)
    #y = Softmax()(y)
    return Model(inputs=x_in, outputs=y)

def get_loop4_model(input_shape=(64, 64, 1)):
    x_in = Input(shape=input_shape)
    x_out_list = []
    MERA_layer = GridMERAin16(kernel_dims=4, bond_dims=2, output_dims=2)
    for i in range(16):
        for j in range(16):
            subx = Lambda(lambda x:x[:, 4*i:4*(i+1),4*j:4*(j+1),:] )(x_in)
            x_out_list.append(MERA_layer(subx))
    x_out = Concatenate(axis=1)(x_out_list)
    x_out = Flatten()(x_out)
    y = Dense(2, activation='softmax')(x_out)
    #x_out = Reshape(target_shape=(8, 8, 1))(x_out)
    #y = GridMERAin16(kernel_dims=8, bond_dims=2, output_dims=2)(x_out)
    #y = Softmax()(y)
    return Model(inputs=x_in, outputs=y)

def get_deep_model(input_shape=(64, 64, 1)):
    x_in = Input(shape=input_shape)
    x_in2_list = []
    for i in range(16):
        for j in range(16):
            subx = Lambda(lambda x:x[:, 4*i:4*(i+1),4*j:4*(j+1),:] )(x_in)
            x_in2_list.append(GridMERAin16(kernel_dims=1, bond_dims=2, output_dims=1)(subx))
    x_in2 = Concatenate(axis=1)(x_in2_list)
    x_in2 = Reshape(target_shape=(16, 16, 1))(x_in2)
    x_out_list = []
    for i in range(4):
        for j in range(4):
            subx = Lambda(lambda x:x[:, 4*i:4*(i+1),4*j:4*(j+1),:] )(x_in2)
            x_out_list.append(GridMERAin16(kernel_dims=1, bond_dims=2, output_dims=1)(subx))
    x_out = Concatenate(axis=1)(x_out_list)
    x_out = Reshape(target_shape=(4, 4, 1))(x_out)
    y = GridMERAin16(kernel_dims=1, bond_dims=2, output_dims=2)(x_out)
    y = Softmax()(y)
    return Model(inputs=x_in, outputs=y)



In [2]:
data_path = '/datacommons/carin/fk43/NeedleinHaystack/mnist64_0/'
train_data=ClutteredMNISTDataset(base_path=data_path, csv_path=os.path.join(data_path, 'train.csv'), data_scaling=1., num_examples=3000, balance=0.5)
val_data = ClutteredMNISTDataset(base_path=data_path, csv_path=os.path.join(data_path, 'val.csv'), data_scaling=1., num_examples=1000, balance=0.5)
test_data = ClutteredMNISTDataset(base_path=data_path, csv_path=os.path.join(data_path, 'test.csv'), data_scaling=1., num_examples=1000, balance=0.5)


train_data = Batcher(train_data, batch_size=batch_size)
val_data = Batcher(val_data, batch_size=batch_size)
test_data = Batcher(test_data, batch_size=batch_size)

In [3]:

MERA_model_64 = get_loop4_model(input_shape=(64, 64, 1))

MERA_model_64.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 4, 4, 1)      0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 4, 4, 1)      0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 4, 4, 1)      0           input_1[0][0]                    
______________________________________________________________________________________________

In [ ]:
learning_rate = 0.0002
# TensorNetwork model
MERA_model_64.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate, clipnorm=0.1), metrics=['accuracy'])
MERA_model_64_hist = MERA_model_64.fit_generator(train_data, validation_data=val_data, epochs=50, verbose=1)

# TN model
MERA_model_64.evaluate_generator(test_data, int(np.ceil(1000/batch_size)), workers = 1)

# tn_loss = MERA_model_64_hist.history['loss']
# tn_acc = MERA_model_64_hist.history['accuracy']


# np.savetxt('loss_MERA64.out', tn_loss, delimiter=',')  
# np.savetxt('acc_MERA64.out', tn_acc, delimiter=',')  


Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 47 steps, validate for 16 steps
Epoch 1/50
47/47 [==============================] - 1028s 22s/step - loss: 0.6934 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/50
47/47 [==============================] - 19s 411ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/50
43/47 [==========================>...] - ETA: 1s - loss: 0.6932 - accuracy: 0.5119

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/hpchome/carin/fk43/anaconda3/envs/tfquant/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-342097c871d2>", line 4, in <module>
    MERA_model_64_hist = MERA_model_64.fit_generator(train_data, validation_data=val_data, epochs=50, verbose=1)
  File "/hpchome/carin/fk43/anaconda3/envs/tfquant/lib/python3.7/site-packages/tensorflow_core/python/util/deprecation.py", line 324, in new_func
    return func(*args, **kwargs)
  File "/hpchome/carin/fk43/anaconda3/envs/tfquant/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 1306, in fit_generator
    initial_epoch=initial_epoch)
  File "/hpchome/carin/fk43/anaconda3/envs/tfquant/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "/hpchome/carin/fk43/anaconda3/en

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/hpchome/carin/fk43/anaconda3/envs/tfquant/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-342097c871d2>", line 4, in <module>
    MERA_model_64_hist = MERA_model_64.fit_generator(train_data, validation_data=val_data, epochs=50, verbose=1)
  File "/hpchome/carin/fk43/anaconda3/envs/tfquant/lib/python3.7/site-packages/tensorflow_core/python/util/deprecation.py", line 324, in new_func
    return func(*args, **kwargs)
  File "/hpchome/carin/fk43/anaconda3/envs/tfquant/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 1306, in fit_generator
    initial_epoch=initial_epoch)
  File "/hpchome/carin/fk43/anaconda3/envs/tfquant/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "/hpchome/carin/fk43/anaconda3/en

In [15]:
Dense = tf.keras.layers.Dense
Conv2D = tf.keras.layers.Conv2D
MaxPooling2D = tf.keras.layers.MaxPooling2D

model = tf.keras.Sequential()
model.add(Conv2D(4, kernel_size=(3,3), activation='relu', name='conv1', input_shape=(64,64,1)))
model.add(Conv2D(4, kernel_size=(3,3), activation='relu', name='conv2'))
model.add(Conv2D(4, kernel_size=(3,3), activation='relu', name='conv3'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
#model.add(Dense(1936, activation='relu', name='fc1'))
model.add(Dense(2, activation='softmax', name='fc2'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 62, 62, 4)         40        
_________________________________________________________________
conv2 (Conv2D)               (None, 60, 60, 4)         148       
_________________________________________________________________
conv3 (Conv2D)               (None, 58, 58, 4)         148       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 29, 29, 4)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 3364)              0         
_________________________________________________________________
fc2 (Dense)                  (None, 2)                 6730      
Total params: 7,066
Trainable params: 7,066
Non-trainable params: 0
____________________________________________________

In [16]:
%%time

# Traditional model
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])
hist = model.fit_generator(train_data, validation_data=val_data, epochs=20, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 47 steps, validate for 16 steps
Epoch 1/20
47/47 [==============================] - 5s 102ms/step - loss: 0.7121 - accuracy: 0.4637 - val_loss: 0.6901 - val_accuracy: 0.5010
Epoch 2/20
47/47 [==============================] - 5s 98ms/step - loss: 0.6961 - accuracy: 0.5060 - val_loss: 0.6892 - val_accuracy: 0.5000
Epoch 3/20
47/47 [==============================] - 4s 92ms/step - loss: 0.6836 - accuracy: 0.5383 - val_loss: 0.6861 - val_accuracy: 0.5010
Epoch 4/20
47/47 [==============================] - 4s 92ms/step - loss: 0.6844 - accuracy: 0.5290 - val_loss: 0.6811 - val_accuracy: 0.5270
Epoch 5/20
47/47 [==============================] - 4s 91ms/step - loss: 0.6722 - accuracy: 0.5860 - val_loss: 0.6780 - val_accuracy: 0.5340
Epoch 6/20
47/47 [==============================] - 4s 91ms/step - loss: 0.6703 - accuracy: 0.5663 - val_loss: 0.6764 - val_accuracy: 0.5220
Epoch 7/20
47/47 [==============================] - 4s 91ms/s

In [47]:
class TNLayer(tf.keras.layers.Layer):
    
    def __init__(self, dim):
        super(TNLayer, self).__init__()
        # Create the variables for the layer.
        self.dim = dim
        self.a_var = tf.Variable(tf.random.normal(shape=(dim, dim, 2), stddev=1.0/dim), name="a", trainable=True)
        self.b_var = tf.Variable(tf.random.normal(shape=(dim, dim, 2), stddev=1.0/dim), name="b", trainable=True)
        self.bias = tf.Variable(tf.zeros(shape=(dim, dim)), name="bias", trainable=True)

    def call(self, inputs):
        # Define the contraction.
        # We break it out so we can parallelize a batch using tf.vectorized_map.
        def f(input_vec, a_var, b_var, bias_var):
            # Reshape to a matrix instead of a vector.
            input_vec = tf.reshape(input_vec, (self.dim, self.dim))

            # Now we create the network.
            a = tn.Node(a_var)
            b = tn.Node(b_var)
            x_node = tn.Node(input_vec)
            a[1] ^ x_node[0]
            b[1] ^ x_node[1]
            a[2] ^ b[2]

            # The TN should now look like this
            #   |     |
            #   a --- b
            #    \   /
            #      x

            # Now we begin the contraction.
            c = a @ x_node
            result = (c @ b).tensor

            # Finally, add bias.
            return result + bias_var

        # To deal with a batch of items, we can use the tf.vectorized_map function.
        # https://www.tensorflow.org/api_docs/python/tf/vectorized_map
        result = tf.vectorized_map(lambda vec: f(vec, self.a_var, self.b_var, self.bias), inputs)
        return tf.nn.relu(tf.reshape(result, (-1, self.dim**2)))
    
from tensorflow.keras.layers import Conv2D, MaxPooling2D

tn_model = tf.keras.Sequential()
tn_model.add(Conv2D(4, kernel_size=(3,3), activation='relu', name='conv1', input_shape=(64,64,1)))
tn_model.add(Conv2D(4, kernel_size=(3,3), activation='relu', name='conv2'))
tn_model.add(Conv2D(4, kernel_size=(3,3), activation='relu', name='conv3'))
tn_model.add(MaxPooling2D(pool_size=(2,2)))
tn_model.add(tf.keras.layers.Flatten())
tn_model.add(TNLayer(58))
tn_model.add(Dense(2, activation='softmax', name='fc2'))
tn_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 62, 62, 4)         40        
_________________________________________________________________
conv2 (Conv2D)               (None, 60, 60, 4)         148       
_________________________________________________________________
conv3 (Conv2D)               (None, 58, 58, 4)         148       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 29, 29, 4)         0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 3364)              0         
_________________________________________________________________
tn_layer_12 (TNLayer)        (None, 3364)              16820     
_________________________________________________________________
fc2 (Dense)                  (None, 2)               

In [48]:
%%time

# TensorNetwork model
tn_model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(0.0002, clipnorm=0.05), metrics=['accuracy'])
tn_hist = tn_model.fit_generator(train_data, validation_data=val_data, epochs=20, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 47 steps, validate for 16 steps
Epoch 1/20
47/47 [==============================] - 6s 131ms/step - loss: 0.6947 - accuracy: 0.4310 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/20
47/47 [==============================] - 4s 92ms/step - loss: 0.6937 - accuracy: 0.4360 - val_loss: 0.6931 - val_accuracy: 0.5030
Epoch 3/20
47/47 [==============================] - 4s 93ms/step - loss: 0.6933 - accuracy: 0.4540 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/20
47/47 [==============================] - 4s 90ms/step - loss: 0.6932 - accuracy: 0.4930 - val_loss: 0.6931 - val_accuracy: 0.5340
Epoch 5/20
47/47 [==============================] - 4s 90ms/step - loss: 0.6933 - accuracy: 0.4710 - val_loss: 0.6931 - val_accuracy: 0.5370
Epoch 6/20
47/47 [==============================] - 4s 89ms/step - loss: 0.6932 - accuracy: 0.4477 - val_loss: 0.6931 - val_accuracy: 0.5370
Epoch 7/20
47/47 [==============================] - 4s 91ms/s

In [44]:
class TNLayer(tf.keras.layers.Layer):
    
    def __init__(self, dim):
        super(TNLayer, self).__init__()
        # Create the variables for the layer.
        self.dim = dim
        self.a_var = tf.Variable(tf.random.normal(shape=(dim, dim, 2), stddev=1.0/dim), name="a", trainable=True)
        self.b_var = tf.Variable(tf.random.normal(shape=(dim, dim, 2), stddev=1.0/dim), name="b", trainable=True)
        self.bias = tf.Variable(tf.zeros(shape=(dim, dim)), name="bias", trainable=True)

    def call(self, inputs):
        # Define the contraction.
        # We break it out so we can parallelize a batch using tf.vectorized_map.
        def f(input_vec, a_var, b_var, bias_var):
            # Reshape to a matrix instead of a vector.
            input_vec = tf.reshape(input_vec, (self.dim, self.dim))

            # Now we create the network.
            a = tn.Node(a_var)
            b = tn.Node(b_var)
            x_node = tn.Node(input_vec)
            a[1] ^ x_node[0]
            b[1] ^ x_node[1]
            a[2] ^ b[2]

            # The TN should now look like this
            #   |     |
            #   a --- b
            #    \   /
            #      x

            # Now we begin the contraction.
            c = a @ x_node
            result = (c @ b).tensor

            # Finally, add bias.
            return result + bias_var

        # To deal with a batch of items, we can use the tf.vectorized_map function.
        # https://www.tensorflow.org/api_docs/python/tf/vectorized_map
        result = tf.vectorized_map(lambda vec: f(vec, self.a_var, self.b_var, self.bias), inputs)
        return tf.nn.relu(tf.reshape(result, (-1, self.dim**2)))
    
from tensorflow.keras.layers import Conv2D, MaxPooling2D

MERA_model = tf.keras.Sequential()
MERA_model.add(Conv2D(4, kernel_size=(3,3), activation='relu', name='conv1', input_shape=(64,64,1)))
MERA_model.add(Conv2D(4, kernel_size=(3,3), activation='relu', name='conv2'))
MERA_model.add(Conv2D(4, kernel_size=(3,3), activation='relu', name='conv3'))
MERA_model.add(MaxPooling2D(pool_size=(2,2)))
MERA_model.add(tf.keras.layers.Flatten())
MERA_model.add(TNLayer(58))
MERA_model.add(Dense(2, activation='softmax', name='fc2'))
MERA_model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 62, 62, 4)         40        
_________________________________________________________________
conv2 (Conv2D)               (None, 60, 60, 4)         148       
_________________________________________________________________
conv3 (Conv2D)               (None, 58, 58, 4)         148       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 29, 29, 4)         0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 3364)              0         
_________________________________________________________________
tn_layer_11 (TNLayer)        (None, 3364)              16820     
_________________________________________________________________
fc2 (Dense)                  (None, 2)               

In [45]:
%%time

# TensorNetwork model
MERA_model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(0.0005, clipnorm=0.1), metrics=['accuracy'])
MERA_hist = MERA_model.fit_generator(train_data, validation_data=val_data, epochs=20, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 47 steps, validate for 16 steps
Epoch 1/20
47/47 [==============================] - 5s 110ms/step - loss: 0.6978 - accuracy: 0.4987 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 2/20
47/47 [==============================] - 4s 88ms/step - loss: 0.6946 - accuracy: 0.3673 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/20
47/47 [==============================] - 4s 92ms/step - loss: 0.6932 - accuracy: 0.4610 - val_loss: 0.6931 - val_accuracy: 0.5070
Epoch 4/20
47/47 [==============================] - 4s 90ms/step - loss: 0.6935 - accuracy: 0.4953 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/20
47/47 [==============================] - 4s 89ms/step - loss: 0.6934 - accuracy: 0.4443 - val_loss: 0.6931 - val_accuracy: 0.5020
Epoch 6/20
47/47 [==============================] - 4s 91ms/step - loss: 0.6936 - accuracy: 0.4597 - val_loss: 0.6931 - val_accuracy: 0.5160
Epoch 7/20
47/47 [==============================] - 4s 89ms/s

In [1]:
loss = hist.history['loss']
acc = hist.history['accuracy']
tn_loss = tn_hist.history['loss']
tn_acc = tn_hist.history['accuracy']
MERA_loss = MERA_hist.history['loss']
MERA_acc = MERA_hist.history['accuracy']

plt.plot(loss, label='Neural Network')
plt.plot(tn_loss, label='Neural Network with Tensor Module')
plt.plot(MERA_loss, label='MERA model')
plt.title('Loss comparsion in training')
#plt.ylim(0, 0.15)
plt.legend()
plt.show()

plt.plot(acc, label='Neural Network')
plt.plot(tn_acc, label='Neural Network with Tensor Module')
plt.plot(MERA_acc, label='MERA model')
plt.title('Accuracy comparsion in training')
plt.legend()
plt.show()

NameError: name 'hist' is not defined